In [1]:
import sklearn
import numpy as np
import pandas as pd

import torch
cuda_available = torch.cuda.is_available()

from simpletransformers.ner import NERModel, NERArgs

# from ner_eval import collect_named_entities
# from ner_eval import compute_metrics
# from ner_eval import compute_precision_recall_wrapper

# from sklearn_crfsuite import metric
from nervaluate import Evaluator

In [3]:
model_name='indobenchmark/indobert-large-p2'

In [4]:
train_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\train.csv")
eval_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\val.csv")
test_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\test.csv")

In [5]:
label_list = [
    "O", 
    "B-PER", "I-PER",
    "B-ORG", "I-ORG",
    "B-LOC", "I-LOC",
    "B-PROD", "I-PROD",
    "B-WA", "I-WA",
    "B-EV", "I-EV",
]

In [6]:
# Create a NERModel
model_args=NERArgs()
# model_args.train_batch_size=64
# model_args.overwrite_output_dir=True
model_args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    # "num_train_epochs": 1,
    "num_train_epochs": 5,
    "weight_decay": 0,
    "learning_rate": 4e-5,

    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": True,

    "do_lower_case": False, #True if using uncased models
}


In [7]:
# model = NERModel('bert', 
#                 'outputs/checkpoint-594-epoch-1',
#                 args=model_args,
#                 labels=label_list,
#                 use_cuda=cuda_available,
#                 )
# # result, model_outputs, wrong_preds = model.eval_model(eval_df)
# # result

In [8]:
model = NERModel('bert',
                model_name,
                args=model_args,
                labels=label_list,
                use_cuda=cuda_available,
                )

Downloading: 100%|██████████| 1.50k/1.50k [00:00<00:00, 1.53MB/s]
Downloading: 100%|██████████| 1.25G/1.25G [01:07<00:00, 19.8MB/s]   
Some weights of BertForTokenClassification were not initialized from the model checkpoint at indobenchmark/indobert-large-p2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Downloading: 100%|██████████| 224k/224k [00:00<00:00, 235kB/s]  
Downloading: 100%|██████████| 112/112 [00:00<00:00, 112kB/s]
Downloading: 100%|██████████| 2.00/2.00 [00:00<00:00, 2.00kB/s]


In [9]:
model.train_model(train_df,  eval_data=eval_df)

2021-12-11 18:59:02 simpletransformers.ner.ner_model INFO:  Converting to features started.
Epoch 1 of 5:   0%|          | 0/5 [00:00<?, ?it/s]d:\ner-idn-tweet\env\lib\site-packages\torch\optim\lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epochs 0/5. Running Loss:    0.3234: 100%|██████████| 594/594 [02:13<00:00,  4.45it/s]
2021-12-11 19:01:30 simpletransformers.ner.ner_model INFO:  Converting to features started.
Epochs 1/5. Running Loss:    0.1095: 100%|██████████| 594/594 [02:13<00:00,  4.47it/s]
2021-12-11 19:04:12 simpletransformers.ner.n

(2970,
 defaultdict(list,
             {'global_step': [594, 1188, 1782, 2000, 2376, 2970],
              'train_loss': [0.32335537672042847,
               0.10951774567365646,
               0.11391420662403107,
               0.08033551275730133,
               0.2100054770708084,
               0.011290247552096844],
              'eval_loss': [0.3291379825505834,
               0.30366841206114564,
               0.3129414471628882,
               0.33186973153844773,
               0.3418945088082512,
               0.3704475175664329],
              'precision': [0.5709169054441261,
               0.599594868332208,
               0.5414560161779576,
               0.5505154639175258,
               0.5591453882230328,
               0.5492443981240229],
              'recall': [0.3521873619089704,
               0.3923994697304463,
               0.4732655766681396,
               0.4719399027839152,
               0.47414935925762264,
               0.4657534246575342],
      

In [10]:
result, model_outputs, y_pred = model.eval_model(test_df)

2021-12-11 19:12:09 simpletransformers.ner.ner_model INFO:  Converting to features started.
Running Evaluation: 100%|██████████| 186/186 [00:07<00:00, 24.55it/s]
2021-12-11 19:12:23 simpletransformers.ner.ner_model INFO: {'eval_loss': 0.19186292326463367, 'precision': 0.7684670373312152, 'recall': 0.6825396825396826, 'f1_score': 0.7229590883616663}


In [11]:
result

{'eval_loss': 0.19186292326463367,
 'precision': 0.7684670373312152,
 'recall': 0.6825396825396826,
 'f1_score': 0.7229590883616663}

In [12]:
df_x_test=test_df.groupby('sentence_id')['words'].apply(list)
x_test=list(df_x_test)

df_y_test=test_df.groupby('sentence_id')['labels'].apply(list)
y_test=list(df_y_test)

In [13]:
predictions, raw_outputs = model.predict(
    # ["@PolsekAlian PolriBerikan inovasi-inovasi Polri yang memudahkan masyarakat, kegiatan pemolisian masyarakat dan seluruh tindakan Kepolisian dalam menghadapi gangguan Kamtibmas, dan lain-lain - Jenderal Polisi Drs. Listyo Sigit Prabowo, https://t.co/f8GZ6VPqPD. - Kapolri #PolriTVRadioPresisi"],
    ["@ShopeeID Bismillah yok menang ���� SHOPEE @ShopeeID #44ShopeeMamamoo #ShopeexMAMAMOO #AdaMamamoodiShopee"], 
    ["@OPPOIndonesia ✍Butuh memori besar 128GB di OPPO A54 buat simpan file-file tugas kuliah dan testimoni online shop -ku ��Semoga oppo A54 bisa menunjang penghasilan keluargaku �� @OPPOIndonesia @nisa_rkt @Bebheey @kus_ica @Erna19_ @Saftriyuni #OPPOA54 https://t.co/bMYIiRb4QF �� 2.921"]
        )

2021-12-11 19:12:23 simpletransformers.ner.ner_model INFO:  Converting to features started.
Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 30.76it/s]


In [14]:
predictions

[[{'@ShopeeID': 'O'},
  {'Bismillah': 'O'},
  {'yok': 'O'},
  {'menang': 'O'},
  {'����': 'O'},
  {'SHOPEE': 'B-PROD'},
  {'@ShopeeID': 'O'},
  {'#44ShopeeMamamoo': 'O'},
  {'#ShopeexMAMAMOO': 'O'},
  {'#AdaMamamoodiShopee': 'O'}]]

In [ ]:
#Now we create the model again using the best estimators

print(sklearn.metrics.classification_report(
    y_test, y_pred, labels=label_list, digits=3
    ))

ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.

In [15]:
evaluator = Evaluator(y_test, y_pred, tags=['LOC', 'PER', 'ORG', 'PROD', 'WA', 'EV'], loader="list")

In [16]:
results, results_per_tag = evaluator.evaluate()

In [17]:
results['partial']

{'correct': 2019,
 'incorrect': 0,
 'partial': 220,
 'missed': 618,
 'spurious': 279,
 'possible': 2857,
 'actual': 2518,
 'precision': 0.8455123113582208,
 'recall': 0.7451872593629681,
 'f1': 0.792186046511628}

In [18]:
results['exact']

{'correct': 2019,
 'incorrect': 220,
 'partial': 0,
 'missed': 618,
 'spurious': 279,
 'possible': 2857,
 'actual': 2518,
 'precision': 0.8018268467037332,
 'recall': 0.7066853342667133,
 'f1': 0.7512558139534884}

In [19]:
with open("./results_1_batch.txt", "w") as f:
    str_results = repr(results)
    f.write(str_results)

In [20]:
with open("./results_per_tag_1_batch.txt", "w") as f:
    str_results_per_tag = repr(results_per_tag)
    f.write(str_results_per_tag)

In [ ]:
import re
for i in range(len(y_test)):
    if len(y_test[i]) != len(y_pred[i]):
        print(x_test[i])
        print(len(x_test[i]))
        print("y_test       ",len(y_test[i]),y_test[i])
        print("y_pred before",len(y_pred[i]),y_pred[i])
        for j in range(len(x_test[i])):
            # if '�' in x_test[i][j]: 
            qmark=re.match(r'^�+$',x_test[i][j])
            if qmark:
                print(qmark)
                y_pred[i].insert(j,'O')
        print("y_pred after ",len(y_pred[i]), y_pred[i])
        print()

['@Yanua02', '@BatuKepla', '@Karsono12732862', '@Andreas50772732', '@AndreMa32495641', '@victorpangkey', '@andriskhas', '@AbyManyuu', '@lizaariani5', '@Nana_Rosaa', '_', '@dwi2dh25', '@BaswirOb', '@Bang__Mus', '@745L023N6', '@Rijani54531379', '@Dewi52502425', '@Celes_being1074', '@Yatimpi97149270', '@begoeloh', '@FEMYLIA_K', '@FKARIN3', '@PRINCESLAMPIR', '@aewin86', '@Elina_Vay', '@amandagempita', '@CynthiaEllenna', '@dchyntiabella', '@Permata_Dewi06', '@Dewianggra91', '@Dewi37492961', '@Putri_Sulita', '@tasya_aa77', '@bunga_lao', '@Rismaliez85', '@SonyaAmriani', '@gituLok', '@hennyindah1404', '@WiwidNurwidaya1', '@OniMeniq74', '@maita_omerta', '@Ek4_Puspita', '@Mahrus30127858', '@Zoer_Riya', '@Reny_soepomo', '@BersamaSahabat4', '@jokowi', '@KPK_RI', '@PerekonomianRI', '@airlangga_hrt', '@KemenkeuRI', 'Infrastruktur', 'adalah', 'fondasi', 'bagi', 'rumah', 'indah', 'Indonesia', 'yg', 'aka', 'dibangun', 'di', 'era', 'pasca', 'Jokowi', 'Jokowi', 'yg', 'bangun', 'fondasi', 'rumahnya', ',',

NameError: name 're' is not defined